# Imports

In [11]:
import os

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Constants

In [5]:
CSV_PATH = '../data/train_v2.csv'

# Main

## One-hoted df

In [18]:
df = pd.read_csv(CSV_PATH)
df.tags = df.tags.apply(lambda x: set(x.split(' ')))
mlb = MultiLabelBinarizer()
encoded = pd.DataFrame(mlb.fit_transform(df.tags), columns=mlb.classes_)
encoded_df = pd.concat((df['image_name'], encoded), axis=1)
encoded_df.to_csv('../data/one-hot.csv', index=False)

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,train_0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,train_1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
2,train_2,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
3,train_3,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,train_4,1,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40474,train_40474,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
40475,train_40475,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
40476,train_40476,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
40477,train_40477,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0
